In [5]:
import numpy as np
import os

In [6]:
base_script_train_pcen = """#!/bin/bash

#SBATCH --job-name={jobname}
#SBATCH --nodes=1
#SBATCH --cpus-per-task=4
#SBATCH --mem=16GB
#SBATCH --time=100:00:00
#SBATCH --mail-user=chris.ick@nyu.edu
#SBATCH --output="outputs/o{outputname}-%j.out"
#SBATCH --export=NONE
#SBATCH --gres=gpu:1

module purge
module load anaconda3/4.3.1
module load sox/intel/14.4.2
module load ffmpeg/intel/3.2.2
module load cuda/10.1.105
module load cudnn/10.1v7.6.5.32

source activate py3.8

SLICES='{slices}'
MODELNAME='{modelname}'
MODELDIR={modeldir}
TRAINFEATURENAMES="{trainfeatures}"

cd /home/ci411/pcen-t-varying/scripts/core_scripts

python 02_train.py --slices=$SLICES --model-id=$MODELNAME --model-dir=$MODELDIR --training-dir="{traindir}" --validation-dir="{valdir}" --feature-names=$TRAINFEATURENAMES --load-pump='/beegfs/ci411/pcen/pumps/pcen'
cd /home/ci411/pcen-t-varying/scripts/{scriptdir}/sbatch_scripts_pcen/
sbatch {eval_script}"""


base_script_eval_pcen = """#!/bin/bash

#SBATCH --job-name={jobname}
#SBATCH --nodes=1
#SBATCH --cpus-per-task=4
#SBATCH --mem=16GB
#SBATCH --time=100:00:00
#SBATCH --mail-user=chris.ick@nyu.edu
#SBATCH --export=NONE
#SBATCH --output="outputs/o{outputname}-%j.out"

module purge
module load anaconda3/4.3.1
module load sox/intel/14.4.2
module load ffmpeg/intel/3.2.2
module load cuda/10.1.105
module load cudnn/10.1v7.6.5.32

source activate py3.8

SLICES='{slices}'
MODELNAME='{modelname}'
MODELDIR={modeldir}
FEATUREDIR='{testdir}'

cd /home/ci411/pcen-t-varying/scripts/core_scripts

python 03_evaluate.py --n-samples=100 --slices=$SLICES --model-id=$MODELNAME --model-dir=$MODELDIR --feature-dir=$FEATUREDIR --feature-names="['URBAN-SED_dry']" --results-name='results_dry'
python 03_evaluate.py --n-samples=100 --slices=$SLICES --model-id=$MODELNAME --model-dir=$MODELDIR --feature-dir=$FEATUREDIR --feature-names="['URBAN-SED_dry','URBAN-SED_sim_short','URBAN-SED_sim_medium','URBAN-SED_sim_long','URBAN-SED_alley','URBAN-SED_bedroom','URBAN-SED_tunnel']" --results-name='results_wet'
python 03_evaluate.py --n-samples=100 --slices=$SLICES --model-id=$MODELNAME --model-dir=$MODELDIR --feature-dir=$FEATUREDIR --feature-names="['URBAN-SED_sim_short']" --results-name='results_short'
python 03_evaluate.py --n-samples=100 --slices=$SLICES --model-id=$MODELNAME --model-dir=$MODELDIR --feature-dir=$FEATUREDIR --feature-names="['URBAN-SED_sim_medium']" --results-name='results_medium'
python 03_evaluate.py --n-samples=100 --slices=$SLICES --model-id=$MODELNAME --model-dir=$MODELDIR --feature-dir=$FEATUREDIR --feature-names="['URBAN-SED_sim_long']" --results-name='results_long'
python 03_evaluate.py --n-samples=100 --slices=$SLICES --model-id=$MODELNAME --model-dir=$MODELDIR --feature-dir=$FEATUREDIR --feature-names="['URBAN-SED_alley']" --results-name='results_alley'
python 03_evaluate.py --n-samples=100 --slices=$SLICES --model-id=$MODELNAME --model-dir=$MODELDIR --feature-dir=$FEATUREDIR --feature-names="['URBAN-SED_bedroom']" --results-name='results_bedroom'
python 03_evaluate.py --n-samples=100 --slices=$SLICES --model-id=$MODELNAME --model-dir=$MODELDIR --feature-dir=$FEATUREDIR --feature-names="['URBAN-SED_tunnel']" --results-name='results_tunnel'

"""

In [7]:
#slice variations
slice_variations = [list(np.arange(n+1)) for n in range(10)]

slice_variations_2 = [list(np.arange(n, 10-n)) for n in range(4,0,-1)]

slice_variations_3 = [[i] for i in range(10)]

slices_all_variations = slice_variations+slice_variations_2+slice_variations_3

In [8]:
model_dir = "'/beegfs/ci411/pcen/models/models_807'"
model_label = '_model807'
script_dir = "models_807"
traindir = "/beegfs/ci411/pcen/features_807/pcen"
unpitchdir = "/beegfs/ci411/pcen/features_807/unpitched/pcen"

trainsets = {'dry_':['URBAN-SED_dry'],\
             'realreverb_':['URBAN-SED_dry', 'URBAN-SED_alley','URBAN-SED_bedroom','URBAN-SED_tunnel'],\
             'simreverb_':['URBAN-SED_dry','URBAN-SED_sim_short','URBAN-SED_sim_medium','URBAN-SED_sim_long'],
             'wet_':['URBAN-SED_dry','URBAN-SED_sim_short','URBAN-SED_sim_medium','URBAN-SED_sim_long',\
                     'URBAN-SED_alley','URBAN-SED_bedroom','URBAN-SED_tunnel']}

for trainset in list(trainsets.keys()):
    for slices in slices_all_variations:
        slice_str = ''.join(map(str, slices))
        name = trainset + slice_str
        modelname = name+model_label
        train_name = 'sbatch_scripts_pcen/train_'+modelname+'.sbatch'
        eval_name = 'sbatch_scripts_pcen/eval_'+modelname+'.sbatch'
        eval_name_short = 'eval_'+modelname+ '.sbatch'
        slice_list_str = str(slices).replace(' ','')
        trainfeatures = trainsets[trainset]
        trainfeatures = str(trainfeatures).replace(' ', '')
        
        train_script = base_script_train_pcen.format(jobname=name, outputname=name, slices=slice_list_str, modelname=modelname,\
                                                        trainfeatures=trainfeatures, modeldir=model_dir, scriptdir=script_dir,\
                                                        eval_script=eval_name_short, traindir=traindir, valdir=unpitchdir)
        eval_script = base_script_eval_pcen.format(jobname=name, outputname=name, slices=slice_list_str, modelname=modelname,\
                                                      trainfeatures=trainfeatures, modeldir=model_dir, testdir=unpitchdir)


        with open(train_name, 'w+') as f:
            f.write(train_script)
            
        with open(eval_name, 'w+') as f:
            f.write(eval_script)

In [9]:
train_scripts = [item for item in os.listdir('./sbatch_scripts_pcen/') if 'train' in item]
print(train_scripts)
with open('./sbatch_scripts_pcen/0run_all.bat', 'w+') as f:
    for script in train_scripts:
        f.write('sbatch {}\n'.format(script))

['train_realreverb_2_model807.sbatch', 'train_simreverb_12345678_model807.sbatch', 'train_wet_5_model807.sbatch', 'train_realreverb_01234_model807.sbatch', 'train_dry_4_model807.sbatch', 'train_realreverb_01234567_model807.sbatch', 'train_wet_234567_model807.sbatch', 'train_simreverb_5_model807.sbatch', 'train_dry_12345678_model807.sbatch', 'train_simreverb_0123456789_model807.sbatch', 'train_wet_0123_model807.sbatch', 'train_wet_4_model807.sbatch', 'train_realreverb_012345_model807.sbatch', 'train_dry_0123_model807.sbatch', 'train_realreverb_012345678_model807.sbatch', 'train_realreverb_3_model807.sbatch', 'train_simreverb_4_model807.sbatch', 'train_simreverb_3456_model807.sbatch', 'train_realreverb_01_model807.sbatch', 'train_dry_5_model807.sbatch', 'train_simreverb_234567_model807.sbatch', 'train_wet_012_model807.sbatch', 'train_wet_9_model807.sbatch', 'train_simreverb_7_model807.sbatch', 'train_dry_012_model807.sbatch', 'train_simreverb_0123456_model807.sbatch', 'train_dry_6_model8

In [10]:
eval_scripts = [item for item in os.listdir('./sbatch_scripts_pcen/') if 'eval' in item]
print(eval_scripts)
with open('./sbatch_scripts_pcen/1eval_all.bat', 'w+') as f:
    for script in eval_scripts:
        f.write('sbatch {}\n'.format(script))

['eval_simreverb_0123456_model807.sbatch', 'eval_dry_0_model807.sbatch', 'eval_realreverb_45_model807.sbatch', 'eval_wet_234567_model807.sbatch', 'eval_simreverb_7_model807.sbatch', 'eval_realreverb_8_model807.sbatch', 'eval_dry_01234567_model807.sbatch', '1eval_all.bat', 'eval_realreverb_0123456789_model807.sbatch', 'eval_wet_1_model807.sbatch', 'eval_simreverb_9_model807.sbatch', 'eval_realreverb_6_model807.sbatch', 'eval_dry_01_model807.sbatch', 'eval_simreverb_01_model807.sbatch', 'eval_simreverb_6_model807.sbatch', 'eval_realreverb_9_model807.sbatch', 'eval_wet_0123456789_model807.sbatch', 'eval_dry_1_model807.sbatch', 'eval_realreverb_0123_model807.sbatch', 'eval_wet_45_model807.sbatch', 'eval_simreverb_8_model807.sbatch', 'eval_simreverb_12345678_model807.sbatch', 'eval_wet_0_model807.sbatch', 'eval_wet_012345678_model807.sbatch', 'eval_wet_01234_model807.sbatch', 'eval_realreverb_7_model807.sbatch', 'eval_dry_0123456789_model807.sbatch', 'eval_realreverb_012345678_model807.sbat

In [11]:
base_script_train_mel = """#!/bin/bash

#SBATCH --job-name={jobname}
#SBATCH --nodes=1
#SBATCH --cpus-per-task=4
#SBATCH --mem=16GB
#SBATCH --time=100:00:00
#SBATCH --mail-user=chris.ick@nyu.edu
#SBATCH --output="outputs/o{outputname}-%j.out"
#SBATCH --export=NONE
#SBATCH --gres=gpu:1

module purge
module load anaconda3/4.3.1
module load sox/intel/14.4.2
module load ffmpeg/intel/3.2.2
module load cuda/10.1.105
module load cudnn/10.1v7.6.5.32

source activate py3.8

MODELNAME='{modelname}'
MODELDIR={modeldir}
TRAINFEATURENAMES="{trainfeatures}"

cd /home/ci411/pcen-t-varying/scripts/core_scripts

python 02_train.py --model-id=$MODELNAME --model-dir=$MODELDIR --training-dir="{traindir}" --validation-dir="{valdir}" --feature-names=$TRAINFEATURENAMES --load-pump='/beegfs/ci411/pcen/pumps/mel'
cd /home/ci411/pcen-t-varying/scripts/{scriptdir}/sbatch_scripts_mel/
sbatch {eval_script}"""


base_script_eval_mel = """#!/bin/bash

#SBATCH --job-name={jobname}
#SBATCH --nodes=1
#SBATCH --cpus-per-task=4
#SBATCH --mem=16GB
#SBATCH --time=100:00:00
#SBATCH --mail-user=chris.ick@nyu.edu
#SBATCH --export=NONE
#SBATCH --output="outputs/o{outputname}-%j.out"

module purge
module load anaconda3/4.3.1
module load sox/intel/14.4.2
module load ffmpeg/intel/3.2.2
module load cuda/10.1.105
module load cudnn/10.1v7.6.5.32


source activate py3.8

MODELNAME='{modelname}'
MODELDIR={modeldir}
FEATUREDIR='{testdir}'

cd /home/ci411/pcen-t-varying/scripts/core_scripts

python 03_evaluate.py --n-samples=100 --model-id=$MODELNAME --model-dir=$MODELDIR --feature-dir=$FEATUREDIR --feature-names="['URBAN-SED_dry']" --results-name='results_dry'
python 03_evaluate.py --n-samples=100 --model-id=$MODELNAME --model-dir=$MODELDIR --feature-dir=$FEATUREDIR --feature-names="['URBAN-SED_dry','URBAN-SED_sim_short','URBAN-SED_sim_medium','URBAN-SED_sim_long','URBAN-SED_alley','URBAN-SED_bedroom','URBAN-SED_tunnel']" --results-name='results_wet'
python 03_evaluate.py --n-samples=100 --model-id=$MODELNAME --model-dir=$MODELDIR --feature-dir=$FEATUREDIR --feature-names="['URBAN-SED_sim_short']" --results-name='results_short'
python 03_evaluate.py --n-samples=100 --model-id=$MODELNAME --model-dir=$MODELDIR --feature-dir=$FEATUREDIR --feature-names="['URBAN-SED_sim_medium']" --results-name='results_medium'
python 03_evaluate.py --n-samples=100 --model-id=$MODELNAME --model-dir=$MODELDIR --feature-dir=$FEATUREDIR --feature-names="['URBAN-SED_sim_long']" --results-name='results_long'
python 03_evaluate.py --n-samples=100 --model-id=$MODELNAME --model-dir=$MODELDIR --feature-dir=$FEATUREDIR --feature-names="['URBAN-SED_alley']" --results-name='results_alley'
python 03_evaluate.py --n-samples=100 --model-id=$MODELNAME --model-dir=$MODELDIR --feature-dir=$FEATUREDIR --feature-names="['URBAN-SED_bedroom']" --results-name='results_bedroom'
python 03_evaluate.py --n-samples=100 --model-id=$MODELNAME --model-dir=$MODELDIR --feature-dir=$FEATUREDIR --feature-names="['URBAN-SED_tunnel']" --results-name='results_tunnel'

"""

In [15]:
model_dir = "'/beegfs/ci411/pcen/models/models_807'"
model_label = '_model807'
script_dir = "models_807"
traindir = "/beegfs/ci411/pcen/features_807/mel"
unpitchdir = "/beegfs/ci411/pcen/features_807/unpitched/mel"

trainsets = {'dry_':['URBAN-SED_dry'],\
             'realreverb_':['URBAN-SED_dry', 'URBAN-SED_alley','URBAN-SED_bedroom','URBAN-SED_tunnel'],\
             'simreverb_':['URBAN-SED_dry','URBAN-SED_sim_short','URBAN-SED_sim_medium','URBAN-SED_sim_long'],
             'wet_':['URBAN-SED_dry','URBAN-SED_sim_short','URBAN-SED_sim_medium','URBAN-SED_sim_long',\
                     'URBAN-SED_alley','URBAN-SED_bedroom','URBAN-SED_tunnel']}

for trainset in list(trainsets.keys()):
    name = trainset + 'mel'
    modelname = name+model_label
    train_name = 'sbatch_scripts_mel/train_'+modelname+'.sbatch'
    eval_name = 'sbatch_scripts_mel/eval_'+modelname+'.sbatch'
    eval_name_short = 'eval_'+modelname+ '.sbatch'
    slice_list_str = str(slices).replace(' ','')
    trainfeatures = trainsets[trainset]
    trainfeatures = str(trainfeatures).replace(' ', '')

    train_script = base_script_train_mel.format(jobname=name, outputname=name, modelname=modelname,\
                                                        trainfeatures=trainfeatures, modeldir=model_dir, scriptdir=script_dir,\
                                                        eval_script=eval_name_short, traindir=traindir, valdir=unpitchdir)
    eval_script = base_script_eval_mel.format(jobname=name, outputname=name, modelname=modelname,\
                                                      trainfeatures=trainfeatures, modeldir=model_dir,\
                                                      testdir=unpitchdir)


    with open(train_name, 'w+') as f:
        f.write(train_script)

    with open(eval_name, 'w+') as f:
        f.write(eval_script)

In [16]:
train_scripts = [item for item in os.listdir('./sbatch_scripts_mel/') if 'train' in item]
print(train_scripts)
with open('./sbatch_scripts_mel/0run_all.bat', 'w+') as f:
    for script in train_scripts:
        f.write('sbatch {}\n'.format(script))

['train_simreverb_mel_model807.sbatch', 'train_realreverb_mel_model807.sbatch', 'train_wet_mel_model807.sbatch', 'train_dry_mel_model807.sbatch']


In [17]:
eval_scripts = [item for item in os.listdir('./sbatch_scripts_mel/') if 'eval' in item]
print(eval_scripts)
with open('./sbatch_scripts_mel/1eval_all.bat', 'w+') as f:
    for script in eval_scripts:
        f.write('sbatch {}\n'.format(script))

['eval_wet_mel_model807.sbatch', 'eval_dry_mel_model807.sbatch', 'eval_simreverb_mel_model807.sbatch', '1eval_all.bat', 'eval_realreverb_mel_model807.sbatch']
